In [1]:
import logging
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Importar Módulos Langchain
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp, CTransformers

# Importar módulo para Front-End Web
import gradio as gr

# Importar módulo para carregar variáveis de ambiente do projeto
from dotenv import load_dotenv
import os

In [106]:
def load_model_and_tokenizer(model_name: str):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        print("Modelo e tokenizador carregados com sucesso.")
        
        # Teste de geração de texto
        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        test_output = generator("Teste de geração de texto.", max_length=50, num_return_sequences=1)
        print("Teste de geração de texto bem-sucedido: ", test_output[0]['generated_text'])
        
        return model, tokenizer
    except Exception as e:
        print(f"Erro ao carregar o modelo ou tokenizador: {e}")
        return None, None

model_name = "tiiuae/falcon-11B"
model, tokenizer = load_model_and_tokenizer(model_name)

tokenizer_config.json:   0%|          | 0.00/3.30k [00:00<?, ?B/s]

c:\Users\micha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\micha\.cache\huggingface\hub\models--tiiuae--falcon-11B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/448 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/2.43G [00:00<?, ?B/s]

: 

In [94]:
class ChatPDF:

    def __init__(self, documents_folder: str, vectordb_folder: str, model_path: str, sentence_embedding_model: str, temperature: float = 0.1):
        """
        Constrói todos os atributos necessários para o objeto ChatPDF

        Args:
            documents_folder (str): Caminho/Pasta com os documentos que serão carregados para conversação no Chat
            vectordb_folder (str): Caminho/Pasta onde ficarão os arquivos do Chroma (Vector DB)
            model_path (str): Caminho/Pasta que aponta para o modelo LLM a ser utilizado
            sentence_embedding_model (str): Nome do modelo de Embedding que será usado para gerar os tokens dos documentos
            temperature (float, optional): Temperatura para calibrar o nível de aleatoriedade das respostas. O padrão é 0.1 (Muito determinístico, pouco aleatório)
        """
        self.documents_folder = documents_folder
        self.vectordb_folder = vectordb_folder
        self.model_path = model_path
        self.sentence_embedding_model = sentence_embedding_model
        self.temperature = temperature
        self.pages = []
        self.chunks = []

    def load(self) -> int:
        """
        Realiza a carga dos documentos do caminho/pasta definido no atributo documents_folder.

        Returns:
            int: Quantidade total de páginas carregadas de todos os arquivos PDF
        """

        loader = DirectoryLoader(
            self.documents_folder,
            glob="**/*.pdf",
            loader_cls=PyMuPDFLoader,
            show_progress=True,
            use_multithreading=True
        )

        self.pages = loader.load()

        return len(self.pages)
    
    def split(self, chunk_size: int = 800, chunk_overlap: int = 350) -> int:
        """
        Realiza o split das páginas em chunks para armazenar no Vector DB

        Args:
            chunk_size (int, optional): Quantidade máxima de caracteres de cada chunk. O padrão é 1500.
            chunk_overlap (int, optional): Quantidade de caracteres de overlap entre chunks. O padrão é 150.

        Returns:
            int: Quantidade total de chunks de todas as páginas de todos os documentos carregados
        """

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )

        self.chunks = text_splitter.split_documents(self.pages)

        return len(self.chunks)
    
    def get_embeddings(self):
        """
        Obtem os embeddings do modelo de linguagem definido no atributo sentence_embedding_model.
        """
        self.embeddings = SentenceTransformerEmbeddings(model_name=self.sentence_embedding_model)

    def store(self):
        """
        Armazena os chunks de todos os documentos no Vector DB, utilizando o embedding definido. 
        """
        vectordb = Chroma.from_documents(
            documents=self.chunks,
            embedding=self.embeddings,
            persist_directory=self.vectordb_folder
        )

        vectordb.persist()

        self.vectordb = vectordb

    def create_llm(self):
        """
        Cria uma LLM local, com base no modelo definido no atributo model_path.
        """
        
        self.llm = LlamaCpp(model_path=self.model_path, verbose=True, n_ctx=2048, temperature=self.temperature)

    def create_retriever(self):
        """
        Cria um retriever de documentos com base do Vector DB já carregado com os documentos.
        """
        self.retriever = self.vectordb.as_retriever(search_kwargs={'k':5})

    def create_qa_session(self):
        """
        Cria uma sessão de QA, usando o LLM e Retriever já instanciados.
        """

        PROMPT_TEMPLATE = """ 
        Você é um assistente especializado em responder perguntas sobre documentos armazenados. 
        Use o contexto fornecido para responder à pergunta de forma clara e concisa. 
        Responda sempre em português.
        Se não souber a resposta, diga que não sabe.
        Contexto: {context}
        Pergunta: {question}
        Resposta:"""

        QA_CHAIN_PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

        self.qa = RetrievalQA.from_chain_type(
            self.llm,
            'stuff',
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={'prompt': QA_CHAIN_PROMPT}
        )

In [95]:
# Carrega as variáveis de ambiente do arquivo .env, para utilizar na hora de parametrizar o objeto ChatPDF
load_dotenv()

MODEL_PATH = os.getenv('MODEL_PATH')
VECTORDB_FOLDER = os.getenv('VECTORDB_FOLDER')
DOCUMENTS_FOLDER = os.getenv('DOCUMENTS_FOLDER')
SENTENCE_EMBEDDING_MODEL = os.getenv('SENTENCE_EMBEDDING_MODEL')

In [96]:
# Cria um objeto ChatPDF
chat = ChatPDF(DOCUMENTS_FOLDER, VECTORDB_FOLDER, MODEL_PATH, SENTENCE_EMBEDDING_MODEL, temperature=0.2)

In [97]:
# Executa a carga dos documentos PDF que iremos interagir na sessão de Chat Q&A
chat.load()

100%|██████████| 2/2 [00:00<00:00, 80.00it/s]


29

In [98]:
# Executa o split dos documentos carregados
chat.split()

32

In [99]:
# Obtem os embeddings do modelo de linguagem selecionado
chat.get_embeddings()

c:\Users\micha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [100]:
# Armazena os chunks dos documentos, junto com o embedding, no Vector DB (Chroma)
chat.store()

In [101]:
# Cria a LLM (LLAMA V2) localmente para interagirmos na sessão de chat
chat.create_llm()

llama_model_loader: loaded meta data with 21 key-value pairs and 364 tensors from D:\Workspace Irede\llamav2_local\models\falcon_11B (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = falcon
llama_model_loader: - kv   1:                               general.name str              = Falcon
llama_model_loader: - kv   2:                      falcon.context_length u32              = 2048
llama_model_loader: - kv   3:                  falcon.tensor_data_layout str              = jploski
llama_model_loader: - kv   4:                    falcon.embedding_length u32              = 4096
llama_model_loader: - kv   5:                 falcon.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                         falcon.block_count u32              = 60
llama_model_loader: - kv   7:                falcon.atten

llm_load_vocab: special tokens cache size = 13
llm_load_vocab: token to piece cache size = 0.3884 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = falcon
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 65024
llm_load_print_meta: n_merges         = 64784
llm_load_print_meta: n_ctx_train      = 2048
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_layer          = 60
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_swa            = 0
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 1.0e-05
llm_load_print_meta: f_norm_rms_eps   = 0.0e+00
llm_load_print_meta: f_clamp_kqv      =

In [102]:
# Cria o retriever, que irá recuperar os documentos do Vector DB, com base nos Prompts e usando a LLM
chat.create_retriever()

In [103]:
# Cria uma sessão de chat para Q&A, com base no LLM e Retriever
chat.create_qa_session()

In [104]:
# Front-End da Aplicação Web - Gradio

chat_history = []

with gr.Blocks() as demo:

    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    chat_history = []
    
    def user(user_message, chat_history):
        
        # Retorna resposta da LLM, através da sessão de Q&A
        result = chat.qa({"query": user_message})
        
        # Realiza um append na tela do chat, contendo a mensagem do usuário e a resposta do modelo
        chat_history.append((user_message, result["result"]))

        return gr.update(value=""), chat_history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.



llama_print_timings:        load time =    1880.93 ms
llama_print_timings:      sample time =      74.37 ms /   256 runs   (    0.29 ms per token,  3442.20 tokens per second)
llama_print_timings: prompt eval time =   46437.56 ms /   341 tokens (  136.18 ms per token,     7.34 tokens per second)
llama_print_timings:        eval time =   80100.34 ms /   255 runs   (  314.12 ms per token,     3.18 tokens per second)
llama_print_timings:       total time =  126902.28 ms /   596 tokens


Keyboard interruption in main thread... closing server.
